In [1]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import pandas as pd
import numpy as np

In [2]:
class InterconnectionScraper:
    def __init__(self):
        self.years = list(range(2015,2024))
    def check_parameter(self,year):
        assert year in self.years, "the year passed was not one of ', '.join(self.years))"
    def make_request(self, year):
        self.check_parameter(year)
        url = f"https://www.energy-charts.info/charts/import_export/chart.htm?l=en&c=DE&year={year}"

        chrome_driver_path = "/opt/homebrew/bin/chromedriver"
        driver = webdriver.Chrome(executable_path=chrome_driver_path)

        driver.get(url)
        soup = BeautifulSoup(driver.page_source,"html5lib")
        
        return soup
    def retrieve_interconnector_flows(self,year):
        soup = self.make_request(year)
        df_country_flows = pd.DataFrame(columns=["From_1","To_1","Value_1","From_2","To_2","Value_2","Year"])

        for row in soup.find_all("path", class_="chord"):
            text = row.text
            text_list = text.split("→")
            from_1 = text_list[0]
            to_1 = text_list[1].split(":")[0]
            value_1 = text_list[1].split(":")[1].split("\n")[0]
            from_2 = text_list[1].split(":")[1].split("\n")[1]
            to_2 = text_list[2].split(":")[0]
            value_2 = text_list[2].split(":")[1]
            year = year

            df_country_flows = df_country_flows.append({"From_1":from_1,"To_1":to_1,"Value_1":value_1,"From_2":from_2,"To_2":to_2,"Value_2":value_2,"Year":year},ignore_index=True)

        return df_country_flows
            

            
        

In [3]:
scraper = InterconnectionScraper()

start_year = 2015
end_year = 2024

years = list(range(start_year,end_year)) 

df_all = pd.DataFrame(columns=["From_1","To_1","Value_1","From_2","To_2","Value_2","Year"])

for i in years:
    df = scraper.retrieve_interconnector_flows(i)
    df_all = df_all.append(df)


/var/folders/g9/8dl_m7ms2gd5gm2zghqh2rl00000gn/T/ipykernel_4986/3608032931.py:11: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=chrome_driver_path)


In [5]:
df_all.head(10)

,From_1,To_1,Value_1,From_2,To_2,Value_2,Year
0,Poland,Czech Republic,358 GWh,Czech Republic,Poland,15 GWh,2015
1,Poland,Germany,869 GWh,Germany,Poland,170 GWh,2015
2,Sweden,Germany,1.9 TWh,Germany,Sweden,152 GWh,2015
3,Sweden,Poland,3.2 TWh,Poland,Sweden,16 GWh,2015
4,Germany,Luxembourg,4.2 TWh,Luxembourg,Germany,22 GWh,2015
5,Austria,Czech Republic,2.7 TWh,Czech Republic,Austria,2.2 TWh,2015
6,Norway,Netherlands,5.8 TWh,Netherlands,Norway,11 GWh,2015
7,Austria,Switzerland,5.2 TWh,Switzerland,Austria,1 TWh,2015
8,Denmark,Germany,5.4 TWh,Germany,Denmark,2.1 TWh,2015
9,Netherlands,Belgium,6.5 TWh,Belgium,Netherlands,1.4 TWh,2015
